# Bayesian Neural Network for Tabular Data

Publication at arXiv:

### Import Required Libraries

In [ ]:
# Libraries for data visualisation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython import display
import os
from PIL import Image
from torch.utils.data.dataset import Dataset

In [ ]:
# PyTorch libraries for neural network model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# Pyro libraries for Bayesian transformation
import pyro
from pyro.distributions import Normal, Categorical
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

In [ ]:
%matplotlib inline

### Loading Data

In [ ]:
# Specify data source
BASE = '/home/odin/Data Science/Temp/PTP/'
TRAIN_FILE = 'df_mr_train_s1k.csv'
TEST_FILE = 'df_mr_test_s1k.csv'

In [ ]:
class TabularDataset(Dataset):
    def __init__(self, PATH):
        xy = np.array(pd.read_csv(PATH, encoding='utf8').values.tolist())
        self.len = len(xy)
        self.x_data = torch.from_numpy(xy[:, :-1].astype(np.float32))
        self.y_data = torch.from_numpy(xy[:,-1].astype(np.int))

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

In [ ]:
train_loader = DataLoader(TabularDataset(BASE+TRAIN_FILE), 
                          batch_size=128, shuffle=True, num_workers=0)

test_loader = DataLoader(TabularDataset(BASE+TEST_FILE), 
                          batch_size=128, shuffle=True, num_workers=0)

## Define Neural Network of the Model

In [ ]:
class NN(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size_1)
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.out = nn.Linear(hidden_size_2, output_size)
        
    def forward(self, x):
        output = self.fc1(x)
        output = self.fc2(output)
        output = F.relu(output)
        output = self.out(output)
        return output

In [ ]:
# Creat the neural network model
net = NN(27, 1024, 512, 2)

## Transfer the model to Bayesian 

In [ ]:
log_softmax = nn.LogSoftmax(dim=1)

In [ ]:
# define function to lift the model to Bayesian representation

def model(x_data, y_data):
    
    fc1w_prior = Normal(loc=torch.zeros_like(net.fc1.weight), scale=torch.ones_like(net.fc1.weight))
    fc1b_prior = Normal(loc=torch.zeros_like(net.fc1.bias), scale=torch.ones_like(net.fc1.bias))

    fc2w_prior = Normal(loc=torch.zeros_like(net.fc2.weight), scale=torch.ones_like(net.fc2.weight))
    fc2b_prior = Normal(loc=torch.zeros_like(net.fc2.bias), scale=torch.ones_like(net.fc2.bias))

    outw_prior = Normal(loc=torch.zeros_like(net.out.weight), scale=torch.ones_like(net.out.weight))
    outb_prior = Normal(loc=torch.zeros_like(net.out.bias), scale=torch.ones_like(net.out.bias))
    
    priors = {'fc1.weight': fc1w_prior, 'fc1.bias': fc1b_prior, 'fc2.weight': fc2w_prior, 'fc2.bias': fc2b_prior, 'out.weight': outw_prior, 'out.bias': outb_prior}
    
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", net, priors)
    
    # sample a regressor (which also samples w and b)
    lifted_reg_model = lifted_module()
    
    lhat = log_softmax(lifted_reg_model(x_data))
    
    pyro.sample("obs", Categorical(logits=lhat), obs=y_data)

In [ ]:
softplus = torch.nn.Softplus()

In [ ]:
# Define guide 

def guide(x_data, y_data):
    
    # First layer weight distribution priors
    fc1w_mu = torch.randn_like(net.fc1.weight)
    fc1w_sigma = torch.randn_like(net.fc1.weight)
    fc1w_mu_param = pyro.param("fc1w_mu", fc1w_mu)
    fc1w_sigma_param = softplus(pyro.param("fc1w_sigma", fc1w_sigma))
    fc1w_prior = Normal(loc=fc1w_mu_param, scale=fc1w_sigma_param)
    
    # First layer bias distribution priors
    fc1b_mu = torch.randn_like(net.fc1.bias)
    fc1b_sigma = torch.randn_like(net.fc1.bias)
    fc1b_mu_param = pyro.param("fc1b_mu", fc1b_mu)
    fc1b_sigma_param = softplus(pyro.param("fc1b_sigma", fc1b_sigma))
    fc1b_prior = Normal(loc=fc1b_mu_param, scale=fc1b_sigma_param)
    
    # Second layer weight distribution priors
    fc2w_mu = torch.randn_like(net.fc2.weight)
    fc2w_sigma = torch.randn_like(net.fc2.weight)
    fc2w_mu_param = pyro.param("fc2w_mu", fc2w_mu)
    fc2w_sigma_param = softplus(pyro.param("fc2w_sigma", fc2w_sigma))
    fc2w_prior = Normal(loc=fc2w_mu_param, scale=fc2w_sigma_param)
    
    # Second layer bias distribution priors
    fc2b_mu = torch.randn_like(net.fc2.bias)
    fc2b_sigma = torch.randn_like(net.fc2.bias)
    fc2b_mu_param = pyro.param("fc2b_mu", fc2b_mu)
    fc2b_sigma_param = softplus(pyro.param("fc2b_sigma", fc2b_sigma))
    fc2b_prior = Normal(loc=fc2b_mu_param, scale=fc2b_sigma_param)
      
    # Output layer weight distribution priors
    outw_mu = torch.randn_like(net.out.weight)
    outw_sigma = torch.randn_like(net.out.weight)
    outw_mu_param = pyro.param("outw_mu", outw_mu)
    outw_sigma_param = softplus(pyro.param("outw_sigma", outw_sigma))
    outw_prior = Normal(loc=outw_mu_param, scale=outw_sigma_param).independent(1)
    
    # Output layer bias distribution priors
    outb_mu = torch.randn_like(net.out.bias)
    outb_sigma = torch.randn_like(net.out.bias)
    outb_mu_param = pyro.param("outb_mu", outb_mu)
    outb_sigma_param = softplus(pyro.param("outb_sigma", outb_sigma))
    outb_prior = Normal(loc=outb_mu_param, scale=outb_sigma_param)
    priors = {'fc1.weight': fc1w_prior, 'fc1.bias': fc1b_prior, 'fc2.weight': fc2w_prior, 'fc2.bias': fc2b_prior, 'out.weight': outw_prior, 'out.bias': outb_prior}
    
    lifted_module = pyro.random_module("module", net, priors)
    
    return lifted_module()

In [ ]:
optim = Adam({"lr": 0.01})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

### Train the model

In [ ]:
num_iterations = 10
loss = 0

for j in range(num_iterations):
    loss = 0
    for batch_id, data in enumerate(train_loader):
        # calculate the loss and take a gradient step
        loss += svi.step(data[0].view(-1,27), data[1])
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = loss / normalizer_train
    
    print("Epoch ", j, " Loss ", total_epoch_loss_train)

## Do prediction (infrence)

In [ ]:
num_samples = 1000
def predict(x):
    sampled_models = [guide(None, None) for _ in range(num_samples)]
    yhats = [model(x).data for model in sampled_models]
    mean = torch.mean(torch.stack(yhats), 0)
    return np.argmax(mean.numpy(), axis=1)

### Prediction when network is forced to predict

In [ ]:
# Prediction when network is forced to predict

print('Prediction when network is forced to predict')
correct = 0
total = 0
count = 0
labels = None
predicted = None
for j, data in enumerate(test_loader):
    images, labels = data # each data image is a row in the table
    predicted = predict(images.view(-1,27))
    total += labels.size(0)
    labels = list(labels)
    #print(predicted)
    #print(labels)
    correct += (predicted == labels).sum().item()
    count += 1
print("accuracy: %d %%" % (100 * correct / total))

#### Measure Performance

In [ ]:
def report_performance(y_hat, y_true):
    import datetime
    from sklearn.metrics import classification_report
    from sklearn.metrics import cohen_kappa_score
    from sklearn.metrics import roc_curve, auc, roc_auc_score


    now = datetime.datetime.now()

    predicted = np.array(y_hat)
    actual = np.array(y_true)

    tp = np.count_nonzero(predicted * actual)
    tn = np.count_nonzero((predicted - 1) * (actual - 1))
    fp = np.count_nonzero(predicted * (actual - 1))
    fn = np.count_nonzero((predicted - 1) * actual)

    accuracy = (tp + tn) / (tp + fp + fn + tn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    fmeasure = (2 * precision * recall) / (precision + recall)
    cohen_kappa_score = cohen_kappa_score(predicted, actual)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(actual, predicted)
    auc_val = auc(false_positive_rate, true_positive_rate)
    roc_auc_val = roc_auc_score(actual, predicted)

    out_string = '=========='+str(now)+'==============\n'
    out_string += 'Strategy:\t' + strategy + '\n'
    out_string += str('Model Name:\t' + model_name+'\n')
    out_string += '-------------------------------------------------' + '\n'

    out_string += 'Total Samples:\t' + str(len(actual)) + '\n'
    out_string += 'Positive Samples:\t' + str(sum(actual)) + '\n'
    out_string += 'Negative Samples:\t' + str(len(actual)-sum(actual)) + '\n'

    out_string += 'True Positive:\t' + str(tp) + '\n'
    out_string += 'True Negative:\t' + str(tn) + '\n'
    out_string += 'False Positive:\t' + str(fp) + '\n'
    out_string += 'False Negative:\t' + str(fn) + '\n'

    out_string += 'Accuracy:\t' + str(accuracy) + '\n'
    out_string += 'Precision:\t' + str(precision) + '\n'
    out_string += 'Recall:\t' + str(recall) + '\n'
    out_string += 'F-measure:\t' + str(fmeasure) + '\n'
    out_string += 'Cohen_Kappa_Score:\t' + str(cohen_kappa_score) + '\n'
    out_string += 'AUC:\t' + str(auc_val) + '\n'
    out_string += 'ROC_AUC:\t' + str(roc_auc_val) + '\n'
    out_string += '\n'
    out_string += classification_report(actual, predicted)
    out_string += '\n'
    print(out_string)
    with open(model_name+'_'+strategy+'_POP.txt', 'a+') as FO:
        FO.write(out_string)

In [ ]:
y_hat = np.array(predicted)
y_true = np.array(labels)
y_hat, y_true
model_name = 'BNN'
strategy = 'Cannot_Refuse'
report_performance(y_hat, y_true)

### Define Fuctionality for Visualisation

In [ ]:
classes = (0, 1)

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    fig, ax = plt.subplots(figsize=(1, 1))
    ax.imshow(npimg,  cmap='gray', interpolation='nearest')
    plt.show()

In [ ]:
num_samples = 1000 # Tune here
def give_uncertainities(x):
    sampled_models = [guide(None, None) for _ in range(num_samples)]
    yhats = [F.log_softmax(model(x.view(-1,27)).data, 1).detach().numpy() for model in sampled_models]
    return np.asarray(yhats)
    #mean = torch.mean(torch.stack(yhats), 0)
    #return np.argmax(mean, axis=1)

In [ ]:
def test_batch(images, labels, plot=True):
    y = give_uncertainities(images)
    predicted_for_images = 0
    correct_predictions=0
    y_pred = []

    for i in range(len(labels)):
    
        if(plot):
            print("Real: ",labels[i].item())
            fig, axs = plt.subplots(1, 2, sharey=True, figsize=(20,2))
    
        all_digits_prob = []
    
        highted_something = False
    
        for j in range(len(classes)):
        
            highlight=False
        
            histo = []
            histo_exp = []
        
            for z in range(y.shape[0]):
                histo.append(y[z][i][j])
                histo_exp.append(np.exp(y[z][i][j]))
            
            prob = np.percentile(histo_exp, 50) #sampling median probability
        
            if(prob>0.7): #select if network thinks this sample is 20% chance of this being a label
                highlight = True #possibly an answer
        
            all_digits_prob.append(prob)
            
            if(plot):
            
                N, bins, patches = axs[j].hist(histo, bins=8, color = "lightgray", lw=0,  weights=np.ones(len(histo)) / len(histo), density=False)
                axs[j].set_title(str(j)+" ("+str(round(prob,2))+")") 
        
            if(highlight):
            
                highted_something = True
                
                if(plot):

                    # We'll color code by height, but you could use any scalar
                    fracs = N / N.max()

                    # we need to normalize the data to 0..1 for the full range of the colormap
                    norm = colors.Normalize(fracs.min(), fracs.max())

                    # Now, we'll loop through our objects and set the color of each accordingly
                    for thisfrac, thispatch in zip(fracs, patches):
                        color = plt.cm.viridis(norm(thisfrac))
                        thispatch.set_facecolor(color)

    
        if(plot):
            plt.show()
    
        predicted = np.argmax(all_digits_prob)
        y_pred.append(predicted)
    
        if(highted_something):
            predicted_for_images+=1
            if(labels[i].item()==predicted):
                if(plot):
                    print("Correct")
                correct_predictions +=1.0
            else:
                if(plot):
                    print("Incorrect :()")
        else:
            if(plot):
                print("Undecided.")
        
        if(plot):
            #imshow(images[i].squeeze())
            pass
        
    
    if(plot):
        print("Summary")
        print("Total images: ",len(labels))
        print("Predicted for: ",predicted_for_images)
        print("Accuracy when predicted: ",correct_predictions/predicted_for_images)
        
    return len(labels), correct_predictions, predicted_for_images, y_pred

### Prediction when network can decide not to predict

In [ ]:
# Prediction when network can decide not to predict

print('Prediction when network can refuse')
correct = 0
total = 0
total_predicted_for = 0
y_hat = []
y_true = []
for j, data in enumerate(test_loader):
    images, labels = data
    
    total_minibatch, correct_minibatch, predictions_minibatch, y_pred_batch = test_batch(images, labels, plot=False)
    total += total_minibatch
    correct += correct_minibatch
    total_predicted_for += predictions_minibatch
    y_hat.extend(y_pred_batch)
    y_true.extend([label.item() for label in labels])

print("Total images: ", total)
print("Skipped: ", total-total_predicted_for)
print("Accuracy when made predictions: %d %%" % (100 * correct / total_predicted_for))



In [ ]:
model_name = 'BNN'
strategy = 'Can_Refuse'
report_performance(y_hat, y_true)

In [ ]:
# preparing for visual evaluation

dataiter = iter(test_loader)
images, labels = dataiter.next()

### For each test sample, let us show a histogram of log-probabilities for each of two classes
Log-probability close to 0 means the probability is close to 1 (exp(0) = 1). When the label that network selected is same as the real label, it shows “Correct”. You can see from histograms that the network has a high uncertainty for both class 0 and 1 when the prediction is incorrect, i.e. where the network is undecided, the distribution of log-probabilities is wide for all classes. In the case of the accurate prediction, you will notice that the distribution for the correct class was narrow while for other class it is wide (which means the network is pretty sure of the correct class). 

In [ ]:
test_batch(images[100:110], labels[100:110], plot=True)